In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [2]:
paper_pmid = 23103169
paper_name = 'qian_zhang_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [7]:
original_data = pd.read_excel('raw_data/mmc2.xls', sheet_name='fitness_combined', skiprows=1)

In [8]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4449 x 22


In [10]:
original_data = original_data[['ORF','YPD rep1 fitness','YPD rep2 fitness','YPG fitness','YPE fitness','SC fitness','OAK fitness','ETH fitness']]

In [11]:
original_data.head()

,ORF,YPD rep1 fitness,YPD rep2 fitness,YPG fitness,YPE fitness,SC fitness,OAK fitness,ETH fitness
0,YAL001C,0.968989,0.990349,0.887079,0.960186,1.013147,0.867756,0.999240
1,YAL002W,0.918527,0.900021,0.923468,0.895325,0.948552,0.159949,0.963964
2,YAL004W,1.008346,1.009375,0.951429,0.960195,1.037183,0.811660,1.005849
3,YAL005C,0.996644,1.003788,0.937373,0.919715,1.030085,0.788691,1.001668
4,YAL007C,0.990941,1.004294,1.012650,1.018167,1.027506,0.947705,1.007461


In [12]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [13]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [14]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, YPD rep1 fitness, YPD rep2 fitness, YPG fitness, YPE fitness, SC fitness, OAK fitness, ETH fitness]
Index: []


In [16]:
original_data['YPD fitness'] = original_data[['YPD rep1 fitness','YPD rep2 fitness']].mean(axis=1)

In [20]:
original_data.drop(columns=['YPD rep1 fitness','YPD rep2 fitness'], inplace=True)

In [18]:
original_data = original_data.groupby('ORF').mean()

# Prepare the final dataset

In [22]:
dataset_ids = [16489, 16488, 16490, 16492, 16491, 16487]

In [23]:
datasets = datasets.reindex(index=dataset_ids)

In [24]:
data = original_data.copy()

In [25]:
data.columns = datasets['name'].values

In [26]:
# Create row index
data.index.name='orf'

In [27]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4424 x 6


# Print out

In [29]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db2 import *

In [31]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [32]:
save_data_to_db(data, paper_pmid)